# Comparison study between hard decision Hamming and using a Chase-2 wrapper

1. Baseline - Raw SNR <--> BER using PAM-4 and Gray coding.
2. Encode and decode with Clause 177-1 68,60 matrix as a single error correcting Hamming code.
3. Using a PAM4 classifier (probabilities), find 2 LPRs (P is a parameter)., and decode using a Chase-2 wrapper



In [18]:
#First introduce the project directory so all python modules could be loaded
import os, sys
projectDir = os.environ.get('IEEE8032DJ')
#You don't have to define an environment variable, but then you have to give a path to the project here
if projectDir == None: 
     projectDir = "c:/users/omer/802.3/"
sys.path.insert(1, projectDir)
reedSolomonProjectDir = os.environ.get('REEDSOLOMON')
if reedSolomonProjectDir == None: 
     reedSolomonProjectDir = "c:/users/omer/reedSolomon/reedSolomon/"
sys.path.insert(1, reedSolomonProjectDir)
import numpy as np
import modulationFunctions as mf
from ieee8023dj_d0p1 import encode_177_5
from hammingBasics import simpleHammingDecoder, hammingWrapper128_68_128, indexToSyndrome, syndromeToIndex
from ieeeConstants import parityMatrix_177_5, g_177_1, IEEE_8023_INT_DATA_TYPE
from comparisonStudy import simpleAWGNComparison
from chase2 import chase2Decoder
# Create a local prng for reproducibility 
localPrng = np.random.RandomState(8023)
# create an instance of clause 177 Hamming encoder with bit pair XOR
SNR_MIN     = 0
SNR_MAX     = 16
MESSAGE_LENGTH = 120
CODEWORD_LENGTH = 128
numberOfCodewords = 10
AVERAGE_POWER_PER_BIT_PAM4 = 1 #Currently 1, as half of epsilon_g which is 2
CORRECTION_FACTOR = 0.5 # This factor is meant to be removed once I figure out the right conversion from linear SNR to noise
PAM4_SER_TO_BER = 3/4 # This needs checking and explaining.
EbN0_dB    = np.arange(SNR_MIN,SNR_MAX+1,1)
EbN0_linear        = 10**(EbN0_dB/10.0)  # linear SNR
BerStatsPam4GrayCoded = np.zeros(len(EbN0_linear))
BerStatsPam4Chase2 = np.zeros(len(EbN0_linear))
BerStatsPam4Hamming = np.zeros(len(EbN0_linear))
LOCAL_PRNG = np.random.RandomState(8023)

In [19]:

# Randomize some binary data (for testiong Hamming alone it's enough to use the all 0 codeword, but not enough for testing LPR + Chase-2)
data = localPrng.randint(0,2, (numberOfCodewords, MESSAGE_LENGTH), dtype = IEEE_8023_INT_DATA_TYPE)

loop = 0
for snr in EbN0_linear:
    numberOfBits = 10000 #Omer:  need to change this to be a function of SNR
    # signal vector, new vector for each SNR value
    data = np.random.randint(low = 0, high = 2, size = numberOfBits)
    #modulatedDataNoGrayCoding, pam4SymbolsNoGrayCoding, _ = mf.modulatePAM4(data, grayCoding=False, precoding=False)
    modulatedDataGrayCoded, pam4SymbolsGrayCoded, _ = mf.modulatePAM4(data, grayCoding=True, precoding=False)
        
    # noise
    #n = np.sqrt(No/2)*np.random.randn(VEC_SIZE)
    noisePam4 = LOCAL_PRNG.normal(0, CORRECTION_FACTOR * np.sqrt(0.5 * (AVERAGE_POWER_PER_BIT_PAM4 / snr)), len(modulatedDataGrayCoded))

    # signal + noise
    noisyDataGrayCoded = modulatedDataGrayCoded + noisePam4
     

    # slice received signal + noise
    #pam4SymbolsNoGrayCoding, error, classifier = mf.pam4Slice(noisyDataNoGrayCoding)
    pam4SymbolsGrayCoded, error, classifier = mf.pam4Slice(noisyDataGrayCoded)
    receivedBits, receivedProbabilities = mf.pam4ClassifierToBits(classifier)
    
    #dataRxNoGrayCoding = mf.pam4SymbolsToBits(pam4SymbolsNoGrayCoding, grayCoded= False)
    dataRxGrayCoded = mf.pam4SymbolsToBits(pam4SymbolsGrayCoded, grayCoded= True)
    
    # Use Hamming on sliced data
    decodedDataHamming = hammingWrapper128_68_128(dataRxGrayCoded)
    # And use Chase-2 with the classifier (soft data)
    decodedDataChase2, score, decoderFailure, results = chase2Decoder(hammingWrapper128_68_128, receivedBits, receivedProbabilities, numberOfLeastProbable = 2)

    # find erroneous symbols
    
    BerStatsPam4GrayCoded[loop] += np.sum(dataRxGrayCoded != data) / VEC_SIZE
    BerStatsPam4Hamming[loop] += np.sum(decodedDataHamming != data) / VEC_SIZE
    BerStatsPam4Chase2[loop] += np.sum(decodedDataChase2 != data) / VEC_SIZE
    loop += 1


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Omer\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Omer\AppData\Local\Temp\ipykernel_22844\3244975815.py", line 31, in <module>
    decodedDataChase2, score, decoderFailure, results = chase2Decoder(hammingWrapper128_68_128, receivedBits, receivedProbabilities, numberOfLeastProbable = 2)
                                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Omer\802.3\chase2.py", line 89, in chase2Decoder
UnboundLocalError: cannot access local variable 'score' where it is not associated with a value

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Omer\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb =